In [15]:
import os
import kagglehub
import pandas as pd

In [16]:
# Download latest version
path = kagglehub.dataset_download("rakeshkapilavai/extrovert-vs-introvert-behavior-data")
print("Path to dataset files:", path)
# Afficher les fichiers présents dans le dossier
print("Fichiers présents dans le dataset :")
for root, dirs, files in os.walk(path):
    for file in files:
        print(os.path.join(root, file))

# Charger les deux fichiers
#personality_datasert = pd.read_csv(os.path.join(path, "personality_datasert.csv"))
personality_dataset_brut = pd.read_csv(os.path.join(path, "personality_dataset.csv"))

Path to dataset files: C:\Users\User\.cache\kagglehub\datasets\rakeshkapilavai\extrovert-vs-introvert-behavior-data\versions\2
Fichiers présents dans le dataset :
C:\Users\User\.cache\kagglehub\datasets\rakeshkapilavai\extrovert-vs-introvert-behavior-data\versions\2\personality_datasert.csv
C:\Users\User\.cache\kagglehub\datasets\rakeshkapilavai\extrovert-vs-introvert-behavior-data\versions\2\personality_dataset.csv


In [17]:
# # Infos générales
# print("\nInfos:")
# personality_datasert.info()
print("\nInfos:")
personality_dataset_brut.info()



Infos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2900 entries, 0 to 2899
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Time_spent_Alone           2837 non-null   float64
 1   Stage_fear                 2827 non-null   object 
 2   Social_event_attendance    2838 non-null   float64
 3   Going_outside              2834 non-null   float64
 4   Drained_after_socializing  2848 non-null   object 
 5   Friends_circle_size        2823 non-null   float64
 6   Post_frequency             2835 non-null   float64
 7   Personality                2900 non-null   object 
dtypes: float64(5), object(3)
memory usage: 181.4+ KB


In [18]:
# Statistiques
# print("\nStatistiques :")
# print(personality_datasert.describe(include='all'))
print("\nStatistiques :")
print(personality_dataset_brut.describe(include='all'))



Statistiques :
        Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
count        2837.000000       2827              2838.000000    2834.000000   
unique               NaN          2                      NaN            NaN   
top                  NaN         No                      NaN            NaN   
freq                 NaN       1417                      NaN            NaN   
mean            4.505816        NaN                 3.963354       3.000000   
std             3.479192        NaN                 2.903827       2.247327   
min             0.000000        NaN                 0.000000       0.000000   
25%             2.000000        NaN                 2.000000       1.000000   
50%             4.000000        NaN                 3.000000       3.000000   
75%             8.000000        NaN                 6.000000       5.000000   
max            11.000000        NaN                10.000000       7.000000   

       Drained_after_socializing  F

In [19]:
# 1. Afficher le total de valeurs manquantes par colonne
print("Valeurs manquantes par colonne :")
print(personality_dataset_brut.isnull().sum())

# 2. Voir le pourcentage de valeurs manquantes (utile pour décider de supprimer/remplir)
print("\nPourcentage de valeurs manquantes :")
print((personality_dataset_brut.isnull().sum() / len(personality_dataset_brut)) * 100)

# 3. Optionnel : afficher les lignes contenant au moins une valeur manquante
print("\nLignes avec des valeurs manquantes :")
print(personality_dataset_brut[personality_dataset_brut.isnull().any(axis=1)])

Valeurs manquantes par colonne :
Time_spent_Alone             63
Stage_fear                   73
Social_event_attendance      62
Going_outside                66
Drained_after_socializing    52
Friends_circle_size          77
Post_frequency               65
Personality                   0
dtype: int64

Pourcentage de valeurs manquantes :
Time_spent_Alone             2.172414
Stage_fear                   2.517241
Social_event_attendance      2.137931
Going_outside                2.275862
Drained_after_socializing    1.793103
Friends_circle_size          2.655172
Post_frequency               2.241379
Personality                  0.000000
dtype: float64

Lignes avec des valeurs manquantes :
      Time_spent_Alone Stage_fear  Social_event_attendance  Going_outside  \
6                  4.0         No                      9.0            NaN   
33                 8.0        Yes                      3.0            3.0   
41                 1.0         No                      4.0            5.0

Analyse initiale des valeurs manquantes — personality_dataset

Résumé :
- 7 colonnes présentent des valeurs manquantes sur un total de 8
- Le taux de valeurs manquantes est faible (entre 1.7 % et 2.7 %), donc :
    ➤ Aucune colonne n’a besoin d’être supprimée
    ➤ Pas nécessaire de supprimer les lignes (~423 lignes concernées sur > 2900)
- La colonne cible "Personality" est complète (0 % de NaN)

Décision :
Remplir les valeurs manquantes de manière contrôlée :
- Pour les colonnes numériques : remplacer par la médiane
- Pour les colonnes catégorielles (Yes/No) : remplacer par la modalité la plus fréquente (mode)

Étapes suivantes :
1. Nettoyage des valeurs manquantes comme ci-dessus
2. Vérification finale de l'absence de NaN
3. Export du dataset nettoyé en .xlsx pour analyse dans Tableau Public
4. Passage à l’analyse exploratoire visuelle (EDA) : 
   - Comparaison Introvert vs Extrovert (moyennes, distributions, corrélations)
   - Création d’un dashboard interactif


In [20]:
# Remplissage des colonnes numériques avec la médiane
num_cols = ['Time_spent_Alone', 'Social_event_attendance', 'Going_outside',
            'Friends_circle_size', 'Post_frequency']

# Pour les colonnes numériques
for col in num_cols:
    median = personality_dataset_brut[col].median()
    personality_dataset_brut[col] = personality_dataset_brut[col].fillna(median)

# Remplissage des colonnes catégorielles avec la valeur la plus fréquente
cat_cols = ['Stage_fear', 'Drained_after_socializing']

# Pour les colonnes catégorielles
for col in cat_cols:
    mode = personality_dataset_brut[col].mode()[0]
    personality_dataset_brut[col] = personality_dataset_brut[col].fillna(mode)

# Vérification que toutes les valeurs manquantes ont été comblées
print("Valeurs manquantes après nettoyage :")
print(personality_dataset_brut.isnull().sum())


Valeurs manquantes après nettoyage :
Time_spent_Alone             0
Stage_fear                   0
Social_event_attendance      0
Going_outside                0
Drained_after_socializing    0
Friends_circle_size          0
Post_frequency               0
Personality                  0
dtype: int64


In [21]:
personality_dataset_brut

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert
...,...,...,...,...,...,...,...,...
2895,3.0,No,7.0,6.0,No,6.0,6.0,Extrovert
2896,3.0,No,8.0,3.0,No,14.0,9.0,Extrovert
2897,4.0,Yes,1.0,1.0,Yes,4.0,0.0,Introvert
2898,11.0,Yes,1.0,3.0,Yes,2.0,0.0,Introvert


In [24]:
# Créer le chemin vers ton dossier cible
home_dir = os.path.expanduser("~")
target_path = os.path.join(home_dir, "Documents", "Porfolio", "Data Analysis", "personality_dataset_clean.xlsx")

# Exporter en Excel
personality_dataset_brut.to_excel(target_path, index=False)

print(f"Export terminé : fichier enregistré ici → {target_path}")


Export terminé : fichier enregistré ici → C:\Users\User\Documents\Porfolio\Data Analysis\personality_dataset_clean.xlsx
